In [12]:
import numpy as np
import pandas as pd
data = pd.read_csv('./user_balance_table.csv', parse_dates=['report_date'])

#解析出年，月，日，周，周天
def add_timestamp(data):
	data['report_date'] = pd.to_datetime(data['report_date'],format='%Y%m%d')
	data['day'] = data['report_date'].dt.day
	data['month'] = data['report_date'].dt.month
	data['year'] = data['report_date'].dt.year
	data['week'] = data['report_date'].dt.week
	data['weekday'] = data['report_date'].dt.weekday
	return data

data = add_timestamp(data)
data

data['weekday'].value_counts()

#提取一个给定起点
def get_total_balance(data, begin):
	df_temp =data.copy()
	df_temp = df_temp.groupby(['report_date'])['total_purchase_amt', 'total_redeem_amt'].sum()
	df_temp.reset_index(inplace=True)
	df_temp = df_temp[(df_temp['report_date']>= begin)]
	return df_temp


total_balance = get_total_balance(data, '2014-03-01')

import datetime

#生成9月1到9月30号的空数据
def generate_test_data(data):
	total_balance = data.copy()
	start = datetime.datetime(2014,9,1)
	end = datetime.datetime(2014,10,1)
	testdata = []
	while start != end:
		temp= [start, np.nan, np.nan]
		testdata.append(temp)
		start =start + datetime.timedelta(days=1)
	testdata = pd.DataFrame(testdata)
	testdata.columns = total_balance.columns
	result = pd.concat([total_balance, testdata], axis=0)
	return result

total_balance = generate_test_data(total_balance)
total_balance

total_balance = add_timestamp(total_balance)
total_balance

temp= total_balance.copy()
total_balance = temp.copy()
#取周天的平均值
weekday_weight = total_balance[['weekday', 'total_purchase_amt', 'total_redeem_amt']].groupby('weekday', as_index=False).mean()
weekday_weight

weekday_weight.columns = ['weekday', 'purchase_weekday', 'redeem_weekday']
weekday_weight['purchase_weekday'] /= np.mean(total_balance['total_purchase_amt'])
weekday_weight['redeem_weekday'] /= np.mean(total_balance['total_redeem_amt'])

total_balance = pd.merge(total_balance, weekday_weight, on='weekday', how = 'left')
total_balance

weekday_count = total_balance[['report_date', 'day', 'weekday']].groupby(['day', 'weekday'], as_index=False).count()
weekday_count

weekday_count = pd.merge(weekday_count, weekday_weight, on = 'weekday')
weekday_count

#取全部的平均值
weekday_count['purchase_weekday'] = weekday_count['purchase_weekday']* weekday_count['report_date']/len(np.unique(total_balance['month']) )
weekday_count

weekday_count['redeem_weekday'] = weekday_count['redeem_weekday']* weekday_count['report_date']/len(np.unique(total_balance['month']) )
weekday_count

day_rate = weekday_count.drop(['weekday', 'report_date'], axis =1).groupby('day', as_index=False).sum()
day_rate


day_mean = total_balance[['day', 'total_purchase_amt', 'total_redeem_amt']].groupby('day', as_index=False).mean()
day_mean

day_base = pd.merge(day_mean, day_rate, on= 'day', how = 'left')
day_base['total_purchase_amt'] /=day_base['purchase_weekday']
day_base['total_redeem_amt'] /=day_base['redeem_weekday']


for index, row in day_base.iterrows():
	if row['day'] == 31:
		break
	day_base.loc[index,'report_date']= pd.to_datetime('2014/09/'+str(int(row['day'])))
day_base

day_base['weekday'] = day_base['report_date'].dt.weekday
day_base
#做预测
day_pred = day_base[['day', 'total_purchase_amt', 'total_redeem_amt', 'report_date', 'weekday']]
day_pred

day_pred = pd.merge(day_pred, weekday_weight, on= 'weekday')
day_pred['total_purchase_amt'] *= day_pred['purchase_weekday']
day_pred['total_redeem_amt'] *= day_pred['redeem_weekday']
day_pred

day_pred = day_pred.sort_values('report_date')[['report_date', 'total_purchase_amt', 'total_redeem_amt']]

day_pred['report_date'] = day_pred['report_date'].apply(lambda x: str(x).replace('-','')[0:8])
#生成文件
day_pred.to_csv('rule_base.csv', index=False, header=None)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
